In [1]:
from datasets import load_dataset
dataset = load_dataset("peiyi9979/Math-Shepherd")

In [2]:
import re
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained('peiyi9979/math-shepherd-mistral-7b-prm')

def remove_and_track_indices(lst, target=12902):
    # remove target and return the index of token before target in new list
    indices_of_target = [i for i, num in enumerate(lst) if num == target]
    adjusted_indices = []
    count = 0  # Counter to keep track of how many targets have been removed

    for index in indices_of_target:
        if index > 0:  # Ensure there is an element before the target
            adjusted_indices.append(index - 1 - count)
        count += 1  # Increment for each target found
    
    lst = [num for num in lst if num != target]  # Create a new list without the target
    return lst, adjusted_indices

def process_texts(example):
    t1, t2 = example['input'],example['label']
    i = 0
    j = 0  # separate index for t2
    result_list = []
    while i < len(t1):
        if t1[i:i+2] == "ки":
            if t2[j] == '+':
                result_list.append(1)
            elif t2[j] == '-':
                result_list.append(0)
            i += 2  # Skip "ки" in t1
            j += 1  # Move past the corresponding '+' or '-' in t2
        else:
            i += 1
            j += 1  # Keep t2 in sync with t1

    s = re.sub(r"Step \d+: ", "", t1) # Step i
    s = re.sub(r"<<[^>]*>>", "", s) # <<80*2=1600>>
    input_id = tokenizer.encode(s)
    input_id,index = remove_and_track_indices(input_id)
    example['input_id'] = input_id
    example['index'] = index
    example['targets'] = result_list
    return example

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
processed_dataset = dataset.map(process_texts, num_proc=4)

Map (num_proc=4):   0%|          | 0/444655 [00:00<?, ? examples/s]

In [4]:
processed_dataset = processed_dataset.remove_columns(['input', 'label', 'task'])

In [11]:
processed_dataset.save_to_disk('../Data/Math-Shepherd')

Saving the dataset (0/2 shards):   0%|          | 0/444655 [00:00<?, ? examples/s]

In [1]:
from datasets import load_dataset
dataset = load_dataset('../Data/Math-Shepherd')

Generating train split: 0 examples [00:00, ? examples/s]